In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import os, random
import cv2

In [2]:
#Loading the data from digit datasets
#data = os.listdir("digits/digits_jpeg/digits_jpeg" )
datapath = os.listdir("digits/digits updated/digits updated" )
data_classes = len(datapath)
data_X = []     
data_y = []

"""
for i in range (0,len(data)):
    data_list = os.listdir("digits/digits_jpeg/digits_jpeg" +"/"+str(i))
    for j in data_list:
        pic = cv2.imread("digits/digits_jpeg/digits_jpeg" +"/"+str(i)+"/"+j)
        pic = cv2.resize(pic,(32,32))
        data_X.append(pic)
        data_y.append(i)
"""

for i in range (0,data_classes):
    data_list = os.listdir("digits/digits updated/digits updated" +"/"+str(i))
    for j in data_list:
        pic = cv2.imread("digits/digits updated/digits updated" +"/"+str(i)+"/"+j)
        pic = cv2.resize(pic,(64,64))
        data_X.append(pic)
        data_y.append(i)

if len(data_X) == len(data_y) :
    print("Total DataUnits = ",len(data_X))
# Labels and images
data_X = np.array(data_X)
data_y = np.array(data_y)
data_X.shape

Total DataUnits =  10160


(10160, 64, 64, 3)

In [3]:
print(data_y)

[0 0 0 ... 9 9 9]


In [4]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from sklearn.model_selection import train_test_split
#Spliting the train validation and test sets
train_X, test_X, train_y, test_y = train_test_split(data_X,data_y,test_size=0.05)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.2)
print("Training Set Shape = ",train_X.shape)
print("Validation Set Shape = ",valid_X.shape)
print("Test Set Shape = ",test_X.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.src.legacy.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
#from keras.preprocessing.image import load_img
from tensorflow.keras.utils import load_img
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image

In [ ]:
def Prep(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #making image grayscale
    img = cv2.equalizeHist(img) #Histogram equalization to enhance contrast
    img = img/255 #normalizing
    return img
train_X = np.array(list(map(Prep, train_X)))
test_X = np.array(list(map(Prep, test_X)))
valid_X= np.array(list(map(Prep, valid_X)))
#Reshaping the images
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2],1)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], test_X.shape[2],1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2],1)
#Augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
datagen.fit(train_X)

In [ ]:
train_y = to_categorical(train_y, data_classes)
test_y = to_categorical(test_y, data_classes)
valid_y = to_categorical(valid_y, data_classes)

In [ ]:
#Creating a Neural Network
model = Sequential()
model.add((Conv2D(60,(5,5),input_shape=(64, 64, 1) ,padding = 'Same' ,activation='relu')))
model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2)))

#model.add(Dropout(0.25))
model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)

In [ ]:
#Compiling the model
NAME = "NumberRecog-CNN"
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon = 1e-08, decay=0.0)
tensorboard = TensorBoard(log_dir="logsV2/{}".format(NAME))

model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

#Fit the model
with tf.device('/GPU:0'):
    history = model.fit(datagen.flow(train_X, train_y, batch_size=32),
                        epochs = 30, 
                        validation_data = (valid_X, valid_y),
                        verbose = 2, 
                        steps_per_epoch= 200,
                        callbacks=[tensorboard])

# Testing the model on the test set
score = model.evaluate(test_X, test_y, verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

Epoch 1/30


In [12]:
model.save('NumModelV1.keras')